In [ ]:
#gcloud config set account '746556916249-computer@developer.gserviceaccount.com'

In [ ]:
import keras
#from keras.models import Sequential
#from keras.layers.core import Dense, Dropout, Activation, Flatten
%from keras.layers.convolutional import Conv2D, MaxPooling2D
#from keras.utils import np_utils

from keras.optimizers import SGD, RMSprop, adam,adadelta
from keras.utils import plot_model,  to_categorical

In [ ]:
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Input
from keras.utils import np_utils
#from keras.datasets import cifar10
from keras import regularizers

In [ ]:
from numpy import *
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
import os
import sys
#import oauth2client.client

In [ ]:
label= pd.read_csv("label3.csv", header=None)

In [ ]:
label.shape

In [ ]:
type(label)

In [ ]:
immatrix= pd.read_csv("Imagee_LungCT28.csv", header=None)

In [ ]:
immatrix.shape

In [ ]:
type(immatrix)

In [ ]:
#from google.cloud import storage

In [ ]:
data,Label = shuffle(immatrix,label, random_state=2)
train_data = [data,Label]

In [ ]:
#train_data

In [ ]:
#batch_size = 1024
nb_classes =15
nb_epoch =200
img_rows, img_cols = 28, 28
img_channels = 1
nb_filters =32
nb_filters2 =64
nb_filters3 =96
nb_filters4 =128
nb_filters5 =256
nb_pool = 2
nb_conv = 3

In [ ]:
(X, y) = (train_data[0],train_data[1])
print(X.shape[0])
print(y.shape[0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
y_train.shape

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.values.reshape(X_test.shape[0], img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /=255
X_test /=255

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print(X_train.shape[0], 'train. samples')
print(X_test.shape[0], 'test. samples')

In [ ]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
Y_train
Y_test

In [ ]:
# Create imput
input_img = Input(shape = (28, 28, 1))

In [ ]:
epochs =100

In [ ]:
# Create Volumes for the Inception module
volume_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)

volume_2 = Conv2D(96, (1,1), padding='same', activation='relu')(input_img)
volume_2 = Conv2D(128, (3,3), padding='same', activation='relu')(volume_2)

volume_3 = Conv2D(16, (1,1), padding='same', activation='relu')(input_img)
volume_3 = Conv2D(32, (5,5), padding='same', activation='relu')(volume_3)

volume_4 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
volume_4 = Conv2D(32, (1,1), padding='same', activation='relu')(volume_4)

# Concatenate all volumes of the Inception module
inception_module = keras.layers.concatenate([volume_1, volume_2, volume_3,
                                             volume_4], axis = 3)

In [ ]:
#input_img2= Input(shape = (256, 256, 3))
input_img2= inception_module


# Create Volumes for the Inception module
volume_11 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img2)

volume_22 = Conv2D(96, (1,1), padding='same', activation='relu')(input_img2)
volume_22 = Conv2D(128, (3,3), padding='same', activation='relu')(volume_22)

volume_33 = Conv2D(16, (1,1), padding='same', activation='relu')(input_img2)
volume_33 = Conv2D(32, (5,5), padding='same', activation='relu')(volume_33)

volume_44 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img2)
volume_44 = Conv2D(32, (1,1), padding='same', activation='relu')(volume_44)

# Concatenate all volumes of the Inception module
inception_module2 = keras.layers.concatenate([volume_11, volume_22, volume_33,                                            
                                               volume_44], axis = 3)

In [ ]:
output = Flatten()(inception_module2)
#output = Flatten()(inception_module)
Dropout(0.4)
#out    = Dense(10, activation='softmax')(output)

In [ ]:
output

In [ ]:
out    = Dense(15, input_dim=64,
                kernel_regularizer=regularizers.l2(0.004),
                activation='softmax')(output)

In [ ]:
out

In [ ]:
model = Model(inputs =input_img, outputs = out)
#model = Model(inputs = input_img, outputs = out)
print(model.summary())

In [ ]:
from keras.optimizers import SGD, Adam

lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=256)

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
import matplotlib.pyplot as plt
history =hist

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy _ Two cascaded single Inception moules')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss -Two cascaded single Inception moules ')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()